# Libraries

In [2]:
from pandas import read_excel


# Load Data

In [3]:
data_file_path = r'data (1).xlsx'

meta_data_df = read_excel(data_file_path,sheet_name=0)
meta_data_df.head()

,Columns,Description,Variable
0,appointmentId,Unique id,Independent
1,inspectionStartTime,inspection date,Independent
2,year,registration year,Independent
3,month,registration month,Independent
4,engineTransmission_battery_value,condition ok: yes\nelse : no,Independent


In [4]:
engine_df = read_excel(data_file_path,sheet_name=1)
engine_df.head()

,appointmentId,inspectionStartTime,year,month,engineTransmission_battery_value,engineTransmission_battery_cc_value_0,engineTransmission_battery_cc_value_1,engineTransmission_battery_cc_value_2,engineTransmission_battery_cc_value_3,engineTransmission_battery_cc_value_4,...,engineTransmission_gearShifting_cc_value_1,engineTransmission_gearShifting_cc_value_2,engineTransmission_comments_value_0,engineTransmission_comments_value_1,engineTransmission_comments_value_2,engineTransmission_comments_value_3,engineTransmission_comments_value_4,fuel_type,odometer_reading,rating_engineTransmission
0,aj_01,2019-02-03 15:43:48,2008,8,No,Weak,NaN,NaN,NaN,NaN,...,NaN,NaN,Pickup Low,Starter Motor / Solenoid malfunctioning,NaN,NaN,NaN,Petrol + CNG,124600,3.0
1,aj_02,2019-01-16 13:02:12,2007,5,Yes,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Petrol + CNG,78108,4.0
2,aj_03,2019-02-09 13:31:04,2012,5,Yes,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Diesel,93348,3.5
3,aj_04,2019-01-18 11:02:53,2013,1,Yes,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Petrol + CNG,48233,3.5
4,aj_05,2019-01-27 12:12:06,2011,7,Yes,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Petrol + CNG,74368,4.0


# EDA

In [5]:
engine_df.describe()

,inspectionStartTime,year,month,engineTransmission_engineOil_cc_value_9,engineTransmission_engine_cc_value_10,odometer_reading,rating_engineTransmission
count,26307,26307.000000,26307.000000,0.0,0.0,26307.000000,26307.000000
mean,2019-02-24 04:17:07.174554624,2010.856578,5.462006,NaN,NaN,76460.143764,3.624663
min,2019-01-02 10:02:34,1989.000000,1.000000,NaN,NaN,1.000000,0.500000
25%,2019-01-29 13:54:15.500000,2008.000000,2.000000,NaN,NaN,46396.000000,3.500000
50%,2019-02-24 12:57:51,2011.000000,5.000000,NaN,NaN,72013.000000,4.000000
75%,2019-03-23 10:11:12.500000,2014.000000,9.000000,NaN,NaN,98289.500000,4.000000
max,2019-04-15 12:47:00,2019.000000,12.000000,NaN,NaN,999999.000000,5.000000
std,NaN,3.766234,3.583866,NaN,NaN,46762.524489,0.847645


# Preprocessing Data

# Model Selection

# Model Training and Evaluation

# Model Fine Tuning

# End